In [1]:

# STEP 1: imports and data download

import numpy as np
import pandas as pd
import yfinance as yf

# List of tickers
tickers = ["SPY", "QQQ", "IWM", "GLD", "TLT"]

# Download daily prices from 2010-01-01
data = yf.download(
    tickers,
    start="2010-01-01",
    auto_adjust=True,    # use adjusted prices
    progress=False
)

# We just need the closing prices
if isinstance(data.columns, pd.MultiIndex):
    prices = data["Close"]
else:
    prices = data

print("Prices shape:", prices.shape)
print(prices.head())


Prices shape: (4002, 5)
Ticker             GLD        IWM        QQQ        SPY        TLT
Date                                                              
2010-01-04  109.800003  51.630493  40.393570  85.279190  57.187565
2010-01-05  109.699997  51.452965  40.393570  85.504967  57.556942
2010-01-06  111.510002  51.404533  40.149906  85.565163  56.786407
2010-01-07  110.820000  51.783817  40.176010  85.926353  56.881931
2010-01-08  111.370003  52.066265  40.506695  86.212280  56.856449


In [13]:

# STEP 2: daily percent returns


# simple % returns: r_t = P_t / P_{t-1} - 1
ret = prices.pct_change().dropna()

print("Return matrix shape:", ret.shape)
print(ret.head())



Return matrix shape: (4001, 5)
Ticker           GLD       IWM       QQQ       SPY       TLT
Date                                                        
2010-01-05 -0.000911 -0.003439  0.000000  0.002647  0.006458
2010-01-06  0.016500 -0.000941 -0.006032  0.000704 -0.013386
2010-01-07 -0.006188  0.007378  0.000650  0.004222  0.001682
2010-01-08  0.004963  0.005454  0.008231  0.003328 -0.000448
2010-01-11  0.013289 -0.004030 -0.004082  0.001397 -0.005487


In [15]:

# STEP 3: time-series momentum backtest


def backtest_ts_mom(ret_df, lookback=60):
    """
    Time-series (trend-following) momentum backtest.

    ret_df : DataFrame of daily returns, rows=dates, cols=assets
    lookback : number of days for momentum signal
    """
    # 1. Time-series momentum signal: rolling sum of past lookback returns
    signal = ret_df.rolling(window=lookback).sum()

    # 2. Convert signal to weights: long if positive, short if negative
    n_assets = ret_df.shape[1]
    weights = np.sign(signal) / n_assets   # each asset at +1/N or -1/N or 0

    # 3. Lag weights by 1 day to avoid look-ahead
    w_lag = weights.shift(1)

    # 4. Portfolio daily returns
    port_ret = (w_lag * ret_df).sum(axis=1).dropna()

    # 5. Performance statistics
    n = len(port_ret)
    mean_daily = port_ret.mean()
    vol_daily  = port_ret.std(ddof=1)
    sharpe_daily = mean_daily / vol_daily if vol_daily != 0 else np.nan

    mean_annual   = mean_daily * 252
    vol_annual    = vol_daily * np.sqrt(252)
    sharpe_annual = sharpe_daily * np.sqrt(252)

    t_stat = mean_daily / (vol_daily / np.sqrt(n)) if vol_daily != 0 else np.nan

    # 6. Drawdown and drawdown duration
    equity = (1 + port_ret).cumprod()
    running_max = equity.cummax()
    drawdown = equity / running_max - 1
    max_dd = drawdown.min()

    underwater = equity < running_max
    max_duration = 0
    current = 0
    for flag in underwater:
        if flag:
            current += 1
            max_duration = max(max_duration, current)
        else:
            current = 0

    stats = {
        "lookback": lookback,
        "n_days": n,
        "mean_daily": mean_daily,
        "vol_daily": vol_daily,
        "sharpe_daily": sharpe_daily,
        "mean_annual": mean_annual,
        "vol_annual": vol_annual,
        "sharpe_annual": sharpe_annual,
        "t_stat": t_stat,
        "max_dd": max_dd,
        "max_dd_days": max_duration,
    }

    return port_ret, stats


In [17]:

# STEP 4: base TS momentum strategy (60-day)


port_ret_60, stats_60 = backtest_ts_mom(ret, lookback=60)

print("\n=== TS MOMENTUM: 60-day lookback ===")
for k, v in stats_60.items():
    print(f"{k:15s}: {v}")



=== TS MOMENTUM: 60-day lookback ===
lookback       : 60
n_days         : 4001
mean_daily     : 0.00013407241314798541
vol_daily      : 0.007384039120172148
sharpe_daily   : 0.018157056181043053
mean_annual    : 0.033786248113292325
vol_annual     : 0.11721798709888612
sharpe_annual  : 0.2882343311764085
t_stat         : 1.148496597877145
max_dd         : -0.20479679984174792
max_dd_days    : 1353


In [19]:

# STEP 5: parameter sweep for lookback


results = []

for lookback in [20, 60, 120, 252]:
    _, stats = backtest_ts_mom(ret, lookback=lookback)
    results.append(stats)

res_df = pd.DataFrame(results)
res_df = res_df[[
    "lookback",
    "sharpe_annual", "mean_annual", "vol_annual",
    "max_dd", "max_dd_days"
]]

print("\n=== TS MOMENTUM PARAMETER SWEEP ===")
print(res_df.sort_values("sharpe_annual", ascending=False))



=== TS MOMENTUM PARAMETER SWEEP ===
   lookback  sharpe_annual  mean_annual  vol_annual    max_dd  max_dd_days
3       252       0.391842     0.039288    0.100266 -0.216295         1375
2       120       0.350973     0.038113    0.108593 -0.201473          633
1        60       0.288234     0.033786    0.117218 -0.204797         1353
0        20       0.281527     0.034567    0.122786 -0.245978         1836


In [21]:
# =============================================================================
# STEP 6: final chosen TS momentum strategy
# =============================================================================

lookback_best = 60   # change this to your best value from the table

port_ret_best, stats_best = backtest_ts_mom(ret, lookback=lookback_best)

print("\n=== FINAL TS MOMENTUM STRATEGY ===")
for k, v in stats_best.items():
    print(f"{k:15s}: {v}")



=== FINAL TS MOMENTUM STRATEGY ===
lookback       : 60
n_days         : 4001
mean_daily     : 0.00013407241314798541
vol_daily      : 0.007384039120172148
sharpe_daily   : 0.018157056181043053
mean_annual    : 0.033786248113292325
vol_annual     : 0.11721798709888612
sharpe_annual  : 0.2882343311764085
t_stat         : 1.148496597877145
max_dd         : -0.20479679984174792
max_dd_days    : 1353
